In [37]:
## Step 1: Mount Google Drive (for accessing dataset and saving the model)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# import Necessary Library
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [40]:
## Step 3: Set Dataset Path from Google Drive
dataset_path = '/content/drive/MyDrive/Diseas'

In [42]:
## Step 4: Data Preprocessing
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,             # Pixel values ko 0-1 range me normalize kiya
    rotation_range=20,          # Image rotate karne ka range 20 degrees
    width_shift_range=0.2,      # Image horizontal shift
    height_shift_range=0.2,     # Image vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Zoom transformation
    horizontal_flip=True,       # Random horizontal flip
    validation_split=0.2        # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
     color_mode='rgb',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
     color_mode='rgb',
    subset='validation'
)

Found 4747 images belonging to 4 classes.
Found 1185 images belonging to 4 classes.


In [48]:
# Step 2: Load Pretrained ResNet-50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = True  # Unfreeze the base model for fine-tuning  # Freeze the base model

In [53]:
# Step 3: Add Custom Classification Layers

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Add dropout to prevent overfitting,
    Dense(train_generator.num_classes, activation='softmax')
])

In [55]:
# Step 4: Compile the Model
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
# Step 5: Train the Model
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model.fit(train_generator, validation_data=val_generator, epochs=15, callbacks=[lr_scheduler])

Epoch 1/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 189s 864ms/step - accuracy: 0.8402 - loss: 0.4301 - val_accuracy: 0.2430 - val_loss: 4.0176 - learning_rate: 5.0000e-04
Epoch 2/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 150s 673ms/step - accuracy: 0.9378 - loss: 0.1860 - val_accuracy: 0.2667 - val_loss: 1.4029 - learning_rate: 5.0000e-04
Epoch 3/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 143s 685ms/step - accuracy: 0.9766 - loss: 0.0711 - val_accuracy: 0.2430 - val_loss: 7.9918 - learning_rate: 5.0000e-04
Epoch 4/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 104s 695ms/step - accuracy: 0.9756 - loss: 0.0850 - val_accuracy: 0.2861 - val_loss: 4.6473 - learning_rate: 5.0000e-04
Epoch 5/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 102s 682ms/step - accuracy: 0.9719 - loss: 0.0761 - val_accuracy: 0.2962 - val_loss: 3.8644 - learning_rate: 5.0000e-04
Epoch 6/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 106s 710ms/step - accuracy: 0.9881 - loss: 0.0445 - val_accuracy: 0.4734 - val_loss: 1.7596 - learning_rate: 2.5000e-04
Epoch 7/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 